<a target="_blank" href="https://drive.google.com/file/d/1M9pli0hbwgpAGWNjP316MWUoktvKaY-0/view?usp=sharing">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 4.4 도로명 데이터 EDA

## 라이브러리 설정

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import plotly.express as px
import plotly.graph_objects as go
import re

## 데이터 불러오기

데이터는 주소기반산업지원서비스에서 제공하는 공개하는 주소 중 도로명 데이터의 2024년 1월 전체자료이다.

깃헙에는 2024년 1월 기준 데이터가 저장되어 있고, 다른 시기의 데이터를 사용하고 싶다면 사이트에서 직접 다운 받을 수 있다.

- [깃헙]()
- [공개하는 주소>도로명](https://business.juso.go.kr/addrlink/attrbDBDwld/attrbDBDwldList.do?cPath=99MD&menu=%EB%8F%84%EB%A1%9C%EB%AA%85)


In [3]:
path = r"./data/TN_SPRD_RDNM.txt"

# 활용가이드에서 확인한 컬럼명 붙여주기
col = ['시군구코드', '도로명번호', '읍면동일련번호', '도로명', '영문도로명', '시도명', '시군구명',
          '읍면동구분', '읍면동코드', '읍면동명', '사용여부', '부여사유',
          '변경이력사유', '변경이력정보', '영문시도명', '영문시군구명', '영문읍면동명',
          '도로구간의시작지점기초번호', '도로구간끝지점기초번호', '효력발생일', '말소일자']

df = pd.read_csv(path, sep='\|', names = col, engine='python', encoding='cp949', dtype=
str, keep_default_na=False, header = None)
df.head()

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,부여사유,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자
0,48250,4805185,00,김해대로2325번길,Gimhae-daero 2325beon-gil,경상남도,김해시,2,,,...,김해대로의 시작지점부터 약 23250m(기초번호 2325번) 지점에서 왼쪽으로 분기...,,,Gyeongsangnam-do,Gimhae-si,,1,46,20091008,
1,48250,4805187,01,김해대로2371번길,Gimhae-daero 2371beon-gil,경상남도,김해시,1,103,부원동,...,김해대로의 시작지점부터 약 23710m(기초번호 2371번) 지점에서 왼쪽으로 분기...,,,Gyeongsangnam-do,Gimhae-si,Buwon-dong,1,30,20091008,
2,48250,4805188,01,김해대로2385번길,Gimhae-daero 2385beon-gil,경상남도,김해시,1,103,부원동,...,김해대로의 시작지점부터 약 23850m(기초번호 2385번) 지점에서 왼쪽으로 분기...,,,Gyeongsangnam-do,Gimhae-si,Buwon-dong,1,48,20091008,
3,48250,4805189,00,김해대로2415번길,Gimhae-daero 2415beon-gil,경상남도,김해시,2,,,...,김해대로의 시작지점부터 약 24150m(기초번호 2415번) 지점에서 왼쪽으로 분기...,,,Gyeongsangnam-do,Gimhae-si,,1,16,20091008,
4,48250,4805190,00,김해대로2431번길,Gimhae-daero 2431beon-gil,경상남도,김해시,2,,,...,김해대로의 시작지점부터 약 24310m(기초번호 2431번) 지점에서 왼쪽으로 분기...,,,Gyeongsangnam-do,Gimhae-si,,1,34,20091008,


In [5]:
df.to_csv('./data/total_road_name_2401.csv', index=False, encoding="utf-8")

In [6]:
# 재실행 시, 이 코드 부터 실행
df = pd.read_csv('./data/total_road_name_2401.csv', encoding='utf-8', dtype='str')
df.head()

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,부여사유,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자
0,48250,4805185,00,김해대로2325번길,Gimhae-daero 2325beon-gil,경상남도,김해시,2,NaN,NaN,...,김해대로의 시작지점부터 약 23250m(기초번호 2325번) 지점에서 왼쪽으로 분기...,NaN,NaN,Gyeongsangnam-do,Gimhae-si,NaN,1,46,20091008,NaN
1,48250,4805187,01,김해대로2371번길,Gimhae-daero 2371beon-gil,경상남도,김해시,1,103,부원동,...,김해대로의 시작지점부터 약 23710m(기초번호 2371번) 지점에서 왼쪽으로 분기...,NaN,NaN,Gyeongsangnam-do,Gimhae-si,Buwon-dong,1,30,20091008,NaN
2,48250,4805188,01,김해대로2385번길,Gimhae-daero 2385beon-gil,경상남도,김해시,1,103,부원동,...,김해대로의 시작지점부터 약 23850m(기초번호 2385번) 지점에서 왼쪽으로 분기...,NaN,NaN,Gyeongsangnam-do,Gimhae-si,Buwon-dong,1,48,20091008,NaN
3,48250,4805189,00,김해대로2415번길,Gimhae-daero 2415beon-gil,경상남도,김해시,2,NaN,NaN,...,김해대로의 시작지점부터 약 24150m(기초번호 2415번) 지점에서 왼쪽으로 분기...,NaN,NaN,Gyeongsangnam-do,Gimhae-si,NaN,1,16,20091008,NaN
4,48250,4805190,00,김해대로2431번길,Gimhae-daero 2431beon-gil,경상남도,김해시,2,NaN,NaN,...,김해대로의 시작지점부터 약 24310m(기초번호 2431번) 지점에서 왼쪽으로 분기...,NaN,NaN,Gyeongsangnam-do,Gimhae-si,NaN,1,34,20091008,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366345 entries, 0 to 366344
Data columns (total 21 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   시군구코드          366345 non-null  object
 1   도로명번호          366345 non-null  object
 2   읍면동일련번호        366345 non-null  object
 3   도로명            366345 non-null  object
 4   영문도로명          366345 non-null  object
 5   시도명            366345 non-null  object
 6   시군구명           363825 non-null  object
 7   읍면동구분          366345 non-null  object
 8   읍면동코드          194617 non-null  object
 9   읍면동명           194617 non-null  object
 10  사용여부           366345 non-null  object
 11  부여사유           364310 non-null  object
 12  변경이력사유         0 non-null       object
 13  변경이력정보         0 non-null       object
 14  영문시도명          366345 non-null  object
 15  영문시군구명         363825 non-null  object
 16  영문읍면동명         194617 non-null  object
 17  도로구간의시작지점기초번호  365071 non-null  object
 18  도로구간

In [8]:
print('총 열 수: ', len(df.columns))
print('총 행 수: ', len(df))
print('중복 제거 후 총 행 수', len(df.drop_duplicates()))

총 열 수:  21
총 행 수:  366345
중복 제거 후 총 행 수 366345


In [9]:
# null이 있는 컬럼 확인
df.isnull().sum()

시군구코드                 0
도로명번호                 0
읍면동일련번호               0
도로명                   0
영문도로명                 0
시도명                   0
시군구명               2520
읍면동구분                 0
읍면동코드            171728
읍면동명             171728
사용여부                  0
부여사유               2035
변경이력사유           366345
변경이력정보           366345
영문시도명                 0
영문시군구명             2520
영문읍면동명           171728
도로구간의시작지점기초번호      1274
도로구간끝지점기초번호        1274
효력발생일              2029
말소일자             366345
dtype: int64

In [ ]:
# 각 컬럼별 유니크 개수 확인
for i in df.columns:
    print(i, len(df[i].unique()))

시군구코드 254
도로명번호 167199
읍면동일련번호 29
도로명 141435
영문도로명 141269
시도명 17
시군구명 232
읍면동구분 3
읍면동코드 122
읍면동명 3969
사용여부 1
부여사유 114674
변경이력사유 1
변경이력정보 1
영문시도명 18
영문시군구명 232
영문읍면동명 3962
도로구간의시작지점기초번호 1133
도로구간끝지점기초번호 3780
효력발생일 2748
말소일자 1


In [ ]:
# 개별 도로명을 구분할 수 있는 고유값 생성하기
df['도로명코드'] = df.apply(lambda x: f"{x['시군구코드']}{x['도로명번호']}", axis=1)
print(len(df['도로명코드'].unique()))
print(len(df))

171728
171728


In [ ]:
pd.DataFrame(df.groupby(['도로명', '도로명코드'])['읍면동코드'].count()).sort_values(by='읍면동코드', ascending=False)

,,읍면동코드
도로명,도로명코드,
퇴계로,111403101011,28
3·15대로,481252019003,22
국채보상로,271103007001,20
제물량로,281103149064,20
율곡로,111103100010,19
...,...,...
에코중앙1로,264403354191,0
에코서로,264403354169,0
에코서9길,264404863121,0


In [ ]:
df.loc[df['도로명코드']=='111403101011']

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
26197,11140,3101011,01,퇴계로,Toegye-ro,서울특별시,중구,1,112,남창동,...,NaN,NaN,Seoul,Jung-gu,Namchang-dong,1,464,20100422,NaN,111403101011
26198,11140,3101011,02,퇴계로,Toegye-ro,서울특별시,중구,1,118,남대문로5가,...,NaN,NaN,Seoul,Jung-gu,Namdaemunno 5(o)-ga,1,464,20100422,NaN,111403101011
26199,11140,3101011,03,퇴계로,Toegye-ro,서울특별시,중구,1,121,회현동1가,...,NaN,NaN,Seoul,Jung-gu,Hoehyeon-dong 1(il)-ga,1,464,20100422,NaN,111403101011
27293,11140,3101011,04,퇴계로,Toegye-ro,서울특별시,중구,1,123,회현동3가,...,NaN,NaN,Seoul,Jung-gu,Hoehyeon-dong 3(sam)-ga,1,464,20100422,NaN,111403101011
27294,11140,3101011,05,퇴계로,Toegye-ro,서울특별시,중구,1,124,충무로1가,...,NaN,NaN,Seoul,Jung-gu,Chungmuro 1(il)-ga,1,464,20100422,NaN,111403101011
27295,11140,3101011,06,퇴계로,Toegye-ro,서울특별시,중구,1,125,충무로2가,...,NaN,NaN,Seoul,Jung-gu,Chungmuro 2(i)-ga,1,464,20100422,NaN,111403101011
27296,11140,3101011,08,퇴계로,Toegye-ro,서울특별시,중구,1,129,남산동2가,...,NaN,NaN,Seoul,Jung-gu,Namsan-dong 2(i)-ga,1,464,20100422,NaN,111403101011
27297,11140,3101011,09,퇴계로,Toegye-ro,서울특별시,중구,1,130,남산동3가,...,NaN,NaN,Seoul,Jung-gu,Namsan-dong 3(sam)-ga,1,464,20100422,NaN,111403101011
27298,11140,3101011,10,퇴계로,Toegye-ro,서울특별시,중구,1,132,충무로4가,...,NaN,NaN,Seoul,Jung-gu,Chungmuro 4(sa)-ga,1,464,20100422,NaN,111403101011
27299,11140,3101011,12,퇴계로,Toegye-ro,서울특별시,중구,1,136,묵정동,...,NaN,NaN,Seoul,Jung-gu,Mukjeong-dong,1,464,20100422,NaN,111403101011


In [ ]:
df.loc[df['도로명코드']=='111403101011']['읍면동명'].unique()

array(['남창동', '남대문로5가', '회현동1가', '회현동3가', '충무로1가', '충무로2가', '남산동2가',
       '남산동3가', '충무로4가', '묵정동', '필동1가', '필동2가', '주자동', '장충동2가', '광희동1가',
       '쌍림동', '을지로7가', '오장동', '신당동', '흥인동', '무학동', nan, '회현동2가', '남산동1가',
       '충무로5가', '남학동', '광희동2가', '충무로3가', '황학동'], dtype=object)

## 데이터 정제하기

In [ ]:
df['읍면동일련번호'].unique()

array(['00', '01', '02', '05', '04', '03', '08', '07', '09', '06', '10',
       '15', '11', '12', '13', '14', '17', '18', '19', '16', '20', '21',
       '22', '24', '25', '26', '28', '23', '27'], dtype=object)

In [ ]:
df = df.loc[df['읍면동일련번호']=='00']
df.reset_index(inplace=True, drop=True)
print(len(df['도로명코드'].unique()))
print(len(df))

171728
171728


In [ ]:
df

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
0,48250,4805185,00,김해대로2325번길,Gimhae-daero 2325beon-gil,경상남도,김해시,2,NaN,NaN,...,NaN,NaN,Gyeongsangnam-do,Gimhae-si,NaN,1,46,20091008,NaN,482504805185
1,48250,4805189,00,김해대로2415번길,Gimhae-daero 2415beon-gil,경상남도,김해시,2,NaN,NaN,...,NaN,NaN,Gyeongsangnam-do,Gimhae-si,NaN,1,16,20091008,NaN,482504805189
2,48250,4805190,00,김해대로2431번길,Gimhae-daero 2431beon-gil,경상남도,김해시,2,NaN,NaN,...,NaN,NaN,Gyeongsangnam-do,Gimhae-si,NaN,1,34,20091008,NaN,482504805190
3,48129,4793411,00,잠도길,Jamdo-gil,경상남도,창원시 진해구,2,NaN,NaN,...,NaN,NaN,Gyeongsangnam-do,"Jinhae-gu, Changwon-si",NaN,1,48,20100621,NaN,481294793411
4,48129,4793412,00,제덕로179번길,Jedeok-ro 179beon-gil,경상남도,창원시 진해구,2,NaN,NaN,...,NaN,NaN,Gyeongsangnam-do,"Jinhae-gu, Changwon-si",NaN,1,42,20100125,NaN,481294793412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171723,44800,4863368,00,역전1길,Yeokjeon 1-gil,충청남도,홍성군,2,NaN,NaN,...,NaN,NaN,Chungcheongnam-do,Hongseong-gun,NaN,1,68,20240129,NaN,448004863368
171724,11680,4863376,00,언주로8길,Eonju-ro 8-gil,서울특별시,강남구,2,NaN,NaN,...,NaN,NaN,Seoul,Gangnam-gu,NaN,1,60,20240126,NaN,116804863376
171725,31110,3354271,00,성안고가도로,Seongangogado-ro,울산광역시,중구,2,NaN,NaN,...,NaN,NaN,Ulsan,Jung-gu,NaN,0,43,20230327,NaN,311103354271
171726,31110,3354272,00,함월고가도로,Hamwolgogado-ro,울산광역시,중구,2,NaN,NaN,...,NaN,NaN,Ulsan,Jung-gu,NaN,0,47,20230327,NaN,311103354272


## 분석1. 중복 도로명 개수

In [ ]:
# 시각화 공통 함수

def graph_viz(df, title):
    fig = px.bar(df,
            x = df.index,
            y = df.columns,
            color = df.index,
            title=f"{title}",
            color_discrete_sequence=px.colors.qualitative.Set2,
            height=400
    )

    fig.update_layout(yaxis={'visible': True, 'showticklabels': False},
                      plot_bgcolor="#F9FAFB",  showlegend=False, paper_bgcolor='#F9FAFB', margin=dict(l=0, r=0, t=40),
                      title={'text': "%s" % title,'y':0.95,'x':0,'xanchor': 'left','yanchor': 'top'},)
    fig.update_xaxes(title=None)
    fig.update_yaxes(title=None, gridcolor='lightgray', mirror=True)
    fig.write_html("./data/%s.html" % title) ## html로 저장
    fig.show()

In [ ]:
dup_road_name = pd.DataFrame(df.groupby('도로명')['도로명코드'].count()).sort_values(by='도로명코드', ascending=False)
dup_road_name = dup_road_name.loc[dup_road_name['도로명코드']>1]
dup_road_name.head(20)

,도로명코드
도로명,
중앙로,93
신촌길,50
신기길,49
향교길,45
신흥길,42
양지길,42
송정길,40
평촌길,39
새터길,39


In [ ]:
print(len(df['도로명'].unique()))
print(len(dup_road_name.index.unique()))
print(len(dup_road_name.index.unique())/len(df['도로명'].unique())*100)

141435
13063
9.236044826245271


In [ ]:
graph_viz(dup_road_name.head(20), "중복 도로명 개수")

In [ ]:
df.loc[df['도로명']=='중앙로']

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
1479,48860,3345047,1,중앙로,Jungang-ro,경상남도,산청군,0,250.0,산청읍,...,NaN,NaN,Gyeongsangnam-do,Sancheong-gun,Sancheong-eup,1,148,20091013.0,NaN,488603345047
3286,47850,3303047,0,중앙로,Jungang-ro,경상북도,칠곡군,2,NaN,NaN,...,NaN,NaN,Gyeongsangbuk-do,Chilgok-gun,NaN,1,350,20080709.0,NaN,478503303047
4965,47250,3303047,3,중앙로,Jungang-ro,경상북도,상주시,1,103.0,인봉동,...,NaN,NaN,Gyeongsangbuk-do,Sangju-si,Inbong-dong,1,392,20090914.0,NaN,472503303047
6335,47170,3303047,1,중앙로,Jungang-ro,경상북도,안동시,1,101.0,삼산동,...,NaN,NaN,Gyeongsangbuk-do,Andong-si,Samsan-dong,1,94,20091027.0,NaN,471703303047
6736,29110,3162073,1,중앙로,Jungang-ro,광주광역시,동구,1,101.0,대인동,...,NaN,NaN,Gwangju,Dong-gu,Daein-dong,71-5,368,20090727.0,NaN,291103162073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164477,52130,3268096,0,중앙로,Jungang-ro,전북특별자치도,군산시,2,NaN,NaN,...,NaN,NaN,Jeonbuk-do,Gunsan-si,NaN,1,264,20100125.0,NaN,521303268096
164831,52790,3279077,0,중앙로,Jungang-ro,전북특별자치도,고창군,2,NaN,NaN,...,NaN,NaN,Jeonbuk-do,Gochang-gun,NaN,1,368,20080905.0,NaN,527903279077
164834,52720,3274056,0,중앙로,Jungang-ro,전북특별자치도,진안군,2,NaN,NaN,...,NaN,NaN,Jeonbuk-do,Jinan-gun,NaN,1,138,20090202.0,NaN,527203274056
168118,52210,3272095,0,중앙로,Jungang-ro,전북특별자치도,김제시,2,NaN,NaN,...,NaN,NaN,Jeonbuk-do,Gimje-si,NaN,1,294,20090626.0,NaN,522103272095


In [ ]:
## 예외: 고속도로
df.loc[df['도로명']=='경부고속도로']

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
127095,41463,1000001,0,경부고속도로,Gyeongbu Expressway,경기도,용인시 기흥구,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,"Giheung-gu, Yongin-si",NaN,387,400,20101228.0,NaN,414631000001
127263,41131,1000001,0,경부고속도로,Gyeongbu Expressway,경기도,성남시 수정구,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,"Sujeong-gu, Seongnam-si",NaN,409,412,20101228.0,NaN,411311000001
128009,11650,1000001,0,경부고속도로,Gyeongbu Expressway,서울특별시,서초구,2,NaN,NaN,...,NaN,NaN,Seoul,Seocho-gu,NaN,411,424,20101228.0,NaN,116501000001
129954,30110,1000001,0,경부고속도로,Gyeongbu Expressway,대전광역시,동구,2,NaN,NaN,...,NaN,NaN,Daejeon,Dong-gu,NaN,265,272,20101228.0,NaN,301101000001
130027,41465,1000001,0,경부고속도로,Gyeongbu Expressway,경기도,용인시 수지구,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,"Suji-gu, Yongin-si",NaN,399,402,20101228.0,NaN,414651000001
130146,41220,1000001,0,경부고속도로,Gyeongbu Expressway,경기도,평택시,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,Pyeongtaek-si,NaN,363,378,20101228.0,NaN,412201000001
130185,41135,1000001,0,경부고속도로,Gyeongbu Expressway,경기도,성남시 분당구,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,"Bundang-gu, Seongnam-si",NaN,401,410,20101228.0,NaN,411351000001
131644,31710,1000001,0,경부고속도로,Gyeongbu Expressway,울산광역시,울주군,2,NaN,NaN,...,NaN,NaN,Ulsan,Ulju-gun,NaN,31,60,20101228.0,NaN,317101000001
131664,30230,1000001,0,경부고속도로,Gyeongbu Expressway,대전광역시,대덕구,2,NaN,NaN,...,NaN,NaN,Daejeon,Daedeok-gu,NaN,1,284,20101228.0,NaN,302301000001
131816,41550,1000001,0,경부고속도로,Gyeongbu Expressway,경기도,안성시,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,Anseong-si,NaN,355,372,20101228.0,NaN,415501000001


##  분석2. 도로유형 기준 도로명 개수

### 고속도로

In [ ]:
df_gsdr = df[df["도로명"].str.endswith("고속도로")]
df_gsdr

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
770,43800,1000023,00,중앙고속도로,Jungang Expressway,충청북도,단양군,2,NaN,NaN,...,NaN,NaN,Chungcheongbuk-do,Danyang-gun,NaN,241,270,20101228,NaN,438001000023
852,48740,1000020,00,중부내륙고속도로,Jungbunaeryuk Expressway,경상남도,창녕군,2,NaN,NaN,...,NaN,NaN,Gyeongsangnam-do,Changnyeong-gun,NaN,17,48,20101228,NaN,487401000020
894,28110,1000029,00,인천국제공항고속도로,Incheongukjegonghang Expressway,인천광역시,중구,2,NaN,NaN,...,NaN,NaN,Incheon,Jung-gu,NaN,1,20,20101228,NaN,281101000029
921,41220,1000007,00,서해안고속도로,Seohaean Expressway,경기도,평택시,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,Pyeongtaek-si,NaN,279,294,20101228,NaN,412201000007
947,44810,1000015,00,당진영덕고속도로,Dangjin Yeongdeok Expressway,충청남도,예산군,2,NaN,NaN,...,NaN,NaN,Chungcheongnam-do,Yesan-gun,NaN,9,44,20101228,NaN,448101000015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169311,52800,1000007,00,서해안고속도로,Seohaean Expressway,전북특별자치도,부안군,2,NaN,NaN,...,NaN,NaN,Jeonbuk-do,Buan-gun,NaN,91,116,20101228,NaN,528001000007
170068,41196,1000028,00,경인고속도로,Gyeongin Expressway,경기도,부천시 오정구,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,"Ojeong-gu, Bucheon-si",NaN,7,14,20180713,NaN,411961000028
170087,41192,1000140,00,수도권제1순환고속도로,Sudogwon Je1sunhwan Expressway,경기도,부천시 원미구,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,"Wonmi-gu, Bucheon-si",NaN,85,90,20201022,NaN,411921000140
170100,41194,1000115,00,수원문산고속도로,Suwon Munsan Expressway,경기도,부천시 소사구,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,"Sosa-gu, Bucheon-si",NaN,31,38,20170714,NaN,411941000115


In [ ]:
print(len(pd.DataFrame(df_gsdr.groupby(['도로명'])['도로명코드'].count()).sort_values(by='도로명코드', ascending=False)))
pd.DataFrame(df_gsdr.groupby(['도로명'])['도로명코드'].count()).sort_values(by='도로명코드', ascending=False)

50


,도로명코드
도로명,
경부고속도로,30
중앙고속도로,21
수도권제1순환고속도로,21
서해안고속도로,19
영동고속도로,17
중부내륙고속도로,15
수원문산고속도로,15
남해고속도로,15
당진영덕고속도로,14


In [ ]:
graph_viz(pd.DataFrame(df_gsdr.groupby(['도로명'])['도로명코드'].count()).sort_values(by='도로명코드', ascending=False), '고속도로 당 부여 도로명코드')

In [ ]:
## 가장 긴 고속도로 이름
max(df_gsdr['도로명'], key=len)

'수도권제2순환고속도로'

### 대로

In [ ]:
df_dr = df[df["도로명"].str.endswith("대로")]
df_dr

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
42,29140,2009004,00,상무대로,Sangmu-daero,광주광역시,서구,2,NaN,NaN,...,NaN,NaN,Gwangju,Seo-gu,NaN,585,1264,20090727,NaN,291402009004
43,29140,2009006,00,죽봉대로,Jukbong-daero,광주광역시,서구,2,NaN,NaN,...,NaN,NaN,Gwangju,Seo-gu,NaN,1,182,20090727,NaN,291402009006
426,44133,2250001,00,동서대로,Dongseo-daero,충청남도,천안시 서북구,2,NaN,NaN,...,NaN,NaN,Chungcheongnam-do,"Seobuk-gu, Cheonan-si",NaN,1,268,20100402,NaN,441332250001
542,48860,3345021,00,석대로,Seokdae-ro,경상남도,산청군,2,NaN,NaN,...,NaN,NaN,Gyeongsangnam-do,Sancheong-gun,NaN,1,548,20091013,NaN,488603345021
818,11170,2005002,00,녹사평대로,Noksapyeong-daero,서울특별시,용산구,2,NaN,NaN,...,NaN,NaN,Seoul,Yongsan-gu,NaN,1,404,20100422,NaN,111702005002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170824,41192,2000015,00,송내대로,Songnae-daero,경기도,부천시 원미구,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,"Wonmi-gu, Bucheon-si",NaN,NaN,NaN,NaN,NaN,411922000015
170825,41194,2000015,00,송내대로,Songnae-daero,경기도,부천시 소사구,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,"Sosa-gu, Bucheon-si",NaN,7,544,20100225,NaN,411942000015
170826,41196,2000015,00,송내대로,Songnae-daero,경기도,부천시 오정구,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,"Ojeong-gu, Bucheon-si",NaN,NaN,NaN,NaN,NaN,411962000015
171610,41196,2000016,00,봉오대로,Bongo-daero,경기도,부천시 오정구,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,"Ojeong-gu, Bucheon-si",NaN,1,562,20090710,NaN,411962000016


In [ ]:
print(len(df_dr['도로명코드'].unique()))
print(len(df_dr))

748
748


In [ ]:
## 가장 긴 대로 이름
max(df_dr['도로명'], key=len)

'테크노중앙대로'

### 로

In [ ]:
gsdril = list(df_gsdr.index) #고속도로에 해당하는 데이터의 인덱스를 리스트로
dril = list(df_dr.index) #대로에 해당하는 데이터의 인덱스를 리스트로
il = gsdril + dril #고속도로와 대로에 해당하는 데이터의 인덱스가 담긴 리스트

In [ ]:
df_ro = df[df["도로명"].str.endswith("로")]
df_ro = df_ro.drop(il) #'로'로 끝나는 데이터 중 고속도로와 대로에 해당하는 데이터를 제외
df_ro

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
44,29140,3000074,00,회재로,Hoejae-ro,광주광역시,서구,2,NaN,NaN,...,NaN,NaN,Gwangju,Seo-gu,NaN,341,1142,20090710,NaN,291403000074
115,44770,3260021,00,신봉시선로,Sinbongsiseon-ro,충청남도,서천군,2,NaN,NaN,...,NaN,NaN,Chungcheongnam-do,Seocheon-gun,NaN,1,206,20110729,NaN,447703260021
116,44770,3260025,00,신화송로,Sinhwasong-ro,충청남도,서천군,2,NaN,NaN,...,NaN,NaN,Chungcheongnam-do,Seocheon-gun,NaN,1,160,20110729,NaN,447703260025
117,44770,3260028,00,장마로,Jangma-ro,충청남도,서천군,2,NaN,NaN,...,NaN,NaN,Chungcheongnam-do,Seocheon-gun,NaN,1,572,20110729,NaN,447703260028
129,44810,3263003,00,가야산로,Gayasan-ro,충청남도,예산군,2,NaN,NaN,...,NaN,NaN,Chungcheongnam-do,Yesan-gun,NaN,1,538,20091113,NaN,448103263003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171711,31110,3354270,00,중구청지하도로,Junggucheongjihado-ro,울산광역시,중구,2,NaN,NaN,...,NaN,NaN,Ulsan,Jung-gu,NaN,0,62,20230327,NaN,311103354270
171716,41820,3354277,00,동고개로,Donggogae-ro,경기도,가평군,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,Gapyeong-gun,NaN,1,72,20240129,NaN,418203354277
171725,31110,3354271,00,성안고가도로,Seongangogado-ro,울산광역시,중구,2,NaN,NaN,...,NaN,NaN,Ulsan,Jung-gu,NaN,0,43,20230327,NaN,311103354271
171726,31110,3354272,00,함월고가도로,Hamwolgogado-ro,울산광역시,중구,2,NaN,NaN,...,NaN,NaN,Ulsan,Jung-gu,NaN,0,47,20230327,NaN,311103354272


In [ ]:
print(len(df_ro))
print(len(df_ro['도로명코드'].unique()))

18423
18423


In [ ]:
# 가장 긴 로 이름
max(df_ro['도로명'], key=len)

'에너지밸리산업11로'

In [ ]:
# 고가로
print(len(df_ro.loc[df_ro['도로명'].str.contains('고가로')]))
## 고가대로
print(len(df_ro.loc[df_ro['도로명'].str.contains('고가도로')]))

1
20


### 길

In [ ]:
df_gil = df[df["도로명"].str.endswith("길")]
df_gil

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
0,48250,4805185,00,김해대로2325번길,Gimhae-daero 2325beon-gil,경상남도,김해시,2,NaN,NaN,...,NaN,NaN,Gyeongsangnam-do,Gimhae-si,NaN,1,46,20091008,NaN,482504805185
1,48250,4805189,00,김해대로2415번길,Gimhae-daero 2415beon-gil,경상남도,김해시,2,NaN,NaN,...,NaN,NaN,Gyeongsangnam-do,Gimhae-si,NaN,1,16,20091008,NaN,482504805189
2,48250,4805190,00,김해대로2431번길,Gimhae-daero 2431beon-gil,경상남도,김해시,2,NaN,NaN,...,NaN,NaN,Gyeongsangnam-do,Gimhae-si,NaN,1,34,20091008,NaN,482504805190
3,48129,4793411,00,잠도길,Jamdo-gil,경상남도,창원시 진해구,2,NaN,NaN,...,NaN,NaN,Gyeongsangnam-do,"Jinhae-gu, Changwon-si",NaN,1,48,20100621,NaN,481294793411
4,48129,4793412,00,제덕로179번길,Jedeok-ro 179beon-gil,경상남도,창원시 진해구,2,NaN,NaN,...,NaN,NaN,Gyeongsangnam-do,"Jinhae-gu, Changwon-si",NaN,1,42,20100125,NaN,481294793412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171720,46890,4863363,00,해양치유길,Haeyangchiyu-gil,전라남도,완도군,2,NaN,NaN,...,NaN,NaN,Jeollanam-do,Wando-gun,NaN,1,240,20240124,NaN,468904863363
171721,11740,4863367,00,아리수로87라길,Arisu-ro 87ra-gil,서울특별시,강동구,2,NaN,NaN,...,NaN,NaN,Seoul,Gangdong-gu,NaN,1,38,20231227,NaN,117404863367
171722,41820,4863360,00,달전1길,Daljeon 1-gil,경기도,가평군,2,NaN,NaN,...,NaN,NaN,Gyeonggi-do,Gapyeong-gun,NaN,1,76,20240129,NaN,418204863360
171723,44800,4863368,00,역전1길,Yeokjeon 1-gil,충청남도,홍성군,2,NaN,NaN,...,NaN,NaN,Chungcheongnam-do,Hongseong-gun,NaN,1,68,20240129,NaN,448004863368


In [ ]:
print(len(df_gil))
print(len(df_gil['도로명코드'].unique()))

152169
152169


In [ ]:
# 가장 긴 길 이름
max(df_gil['도로명'], key=len)

'녹산산단382로14번가길'

### 기타

In [ ]:
len(df) - (len(df_dr) + len(df_gsdr) + len(df_ro) + len(df_gil))

15

In [ ]:
etc = df[~df.isin(df_dr.to_dict(orient='list')).all(axis=1) &
                   ~df.isin(df_gsdr.to_dict(orient='list')).all(axis=1) &
                   ~df.isin(df_ro.to_dict(orient='list')).all(axis=1) &
                   ~df.isin(df_gil.to_dict(orient='list')).all(axis=1)]

etc

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
26769,31110,4307401,00,젊음의거리,Jeolmeumui-geori,울산광역시,중구,2,NaN,NaN,...,NaN,NaN,Ulsan,Jung-gu,NaN,1,114,20080128,NaN,311104307401
30853,31110,4307257,00,새즈믄해거리,Saejeumeunhae-geori,울산광역시,중구,2,NaN,NaN,...,NaN,NaN,Ulsan,Jung-gu,NaN,1,54,20001002,NaN,311104307257
31113,31110,4307170,00,먹자거리,Meokja-geori,울산광역시,중구,2,NaN,NaN,...,NaN,NaN,Ulsan,Jung-gu,NaN,1,40,20001002,NaN,311104307170
35448,31110,4307305,00,시계탑거리,Sigyetap-geori,울산광역시,중구,2,NaN,NaN,...,NaN,NaN,Ulsan,Jung-gu,NaN,1,52,20001002,NaN,311104307305
35604,31110,4307010,00,계변고개,Gyebyeongogae,울산광역시,중구,2,NaN,NaN,...,NaN,NaN,Ulsan,Jung-gu,NaN,1,143,20001002,NaN,311104307010
35611,31110,4307168,00,만남의거리,Mannamui-geori,울산광역시,중구,2,NaN,NaN,...,NaN,NaN,Ulsan,Jung-gu,NaN,1,40,20001002,NaN,311104307168
38179,31110,4307182,00,문화의거리,Munhwaui-geori,울산광역시,중구,2,NaN,NaN,...,NaN,NaN,Ulsan,Jung-gu,NaN,1,44,20001002,NaN,311104307182
102642,31110,4307398,00,젊음의1거리,Jeolmeumui 1-geori,울산광역시,중구,2,NaN,NaN,...,NaN,NaN,Ulsan,Jung-gu,NaN,1,46,20080128,NaN,311104307398
102643,31110,4307399,00,젊음의2거리,Jeolmeumui 2-geori,울산광역시,중구,2,NaN,NaN,...,NaN,NaN,Ulsan,Jung-gu,NaN,1,40,20080128,NaN,311104307399
102644,31110,4307400,00,젊음의3거리,Jeolmeumui 3-geori,울산광역시,중구,2,NaN,NaN,...,NaN,NaN,Ulsan,Jung-gu,NaN,1,26,20110131,NaN,311104307400


### 유형별 수량 비교

In [ ]:
# 중복 고속도로 제외
df_gsdr = df_gsdr.drop_duplicates(subset=['도로명번호', '도로명'], keep='first').copy()
len(df_gsdr)

df_type_total = pd.DataFrame({ '유형' : ['고속도로', '대로', '길', '로', '기타'],
                               '개수' : [len(df_gsdr), len(df_dr), len(df_gil), len(df_ro), len(etc)]})

df_type_total.index = df_type_total['유형']
df_type_total.drop(columns="유형", inplace=True)
df_type_total

,개수
유형,
고속도로,50
대로,748
길,152169
로,18423
기타,15


In [ ]:
import plotly.graph_objects as go

labels = df_type_total.index
values = df_type_total['개수']

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
fig.update_traces(marker=dict(colors=px.colors.qualitative.Set2))
fig.update_layout(title_text="도로유형별 도로명 수량", plot_bgcolor="#F9FAFB", paper_bgcolor='#F9FAFB')
fig.write_html("%s.html" % '도로유형별 도로명 수량')
fig.show()

## 분석3. 도로명 식별의 한계점

### 도로명은 고유한가?

In [ ]:
# 분석 1 과 동일

### 도로명번호는 고유한가?

In [ ]:
# 도로명번호가 2개 이상인 행만 남기기
tmp = df[df.duplicated(subset=['도로명번호'], keep=False)].sort_values(by='도로명번호')
# 도로명번호와 도로명이 동일한 행은 하나만 남긴 후(동일한 도로명, 도로명번호를 갖는 경우 제외하기), 도로명번호로 groupby
df_road_name_unique = pd.DataFrame(tmp.drop_duplicates(subset=['도로명번호', '도로명'], keep='first').groupby('도로명번호').count()).sort_values(by='도로명', ascending=False)
df_road_name_unique.loc[df_road_name_unique['도로명코드']>1]

,시군구코드,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,사용여부,...,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
도로명번호,,,,,,,,,,,,,,,,,,,,,
3265076,3,3,3,3,3,3,3,0,0,3,...,0,0,3,3,0,3,3,3,0,3
3295055,2,2,2,2,2,2,2,0,0,2,...,0,0,2,2,0,2,2,2,0,2
3293063,2,2,2,2,2,2,2,0,0,2,...,0,0,2,2,0,2,2,2,0,2
4730238,2,2,2,2,2,2,2,0,0,2,...,0,0,2,2,0,2,2,2,0,2
4730237,2,2,2,2,2,2,2,0,0,2,...,0,0,2,2,0,2,2,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4838110,2,2,2,2,2,2,2,0,0,2,...,0,0,2,2,0,2,2,2,0,2
4838105,2,2,2,2,2,2,2,0,0,2,...,0,0,2,2,0,2,2,2,0,2
4838104,2,2,2,2,2,2,2,0,0,2,...,0,0,2,2,0,2,2,2,0,2


In [ ]:
df.loc[df['도로명번호']=='3265076']

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
105885,43150,3265076,00,제2바이오밸리로,Je2 bio valley-ro,충청북도,제천시,2,NaN,NaN,...,NaN,NaN,Chungcheongbuk-do,Jecheon-si,NaN,1,120,20111012,NaN,431503265076
120986,44270,3265076,00,옥수로,Oksu-ro,충청남도,당진시,2,NaN,NaN,...,NaN,NaN,Chungcheongnam-do,Dangjin-si,NaN,1,420,20120101,NaN,442703265076
138148,44210,3265076,00,정자동1로,Jeongjadong 1-ro,충청남도,서산시,2,NaN,NaN,...,NaN,NaN,Chungcheongnam-do,Seosan-si,NaN,1,112,20121213,NaN,442103265076


### 도로명코드는 고유한가?

In [ ]:
df[df.duplicated(subset=['도로명번호', '도로명코드'], keep=False)].sort_values(by=['도로명번호', '도로명코드'], ascending=False)

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드


#### 도로명코드의 한계점

In [ ]:
# 도로명, 도로명번호는 같지만 도로명코드가 다른 경우 + 시군구코드가 다른 경우
df[df.duplicated(subset=['도로명', '도로명번호'], keep=False)].groupby(['도로명', '도로명번호']).count().sort_values(by='도로명코드', ascending=False)

,,시군구코드,읍면동일련번호,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,사용여부,부여사유,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
도로명,도로명번호,,,,,,,,,,,,,,,,,,,,
경부고속도로,1000001,30,30,30,30,30,30,0,0,30,29,0,0,30,30,0,29,29,29,0,30
중앙고속도로,1000023,21,21,21,21,21,21,0,0,21,21,0,0,21,21,0,21,21,21,0,21
수도권제1순환고속도로,1000140,21,21,21,21,21,21,0,0,21,20,0,0,21,21,0,20,20,20,0,21
서해안고속도로,1000007,19,19,19,19,19,19,0,0,19,19,0,0,19,19,0,19,19,19,0,19
낙동강자전거길,4859665,18,18,18,18,18,18,0,0,18,18,0,0,18,18,0,18,18,18,0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
병풍로,3017024,2,2,2,2,2,2,0,0,2,2,0,0,2,2,0,2,2,2,0,2
보개원삼로,3012053,2,2,2,2,2,2,0,0,2,2,0,0,2,2,0,2,2,2,0,2
보광로,3012032,2,2,2,2,2,2,0,0,2,2,0,0,2,2,0,2,2,2,0,2


In [ ]:
df.loc[df['도로명번호']=='3253099']

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
57781,44200,3253099,00,희망로,Huimang-ro,충청남도,아산시,2,NaN,NaN,...,NaN,NaN,Chungcheongnam-do,Asan-si,NaN,1,162,20090825,NaN,442003253099
138768,44133,3253099,00,희망로,Huimang-ro,충청남도,천안시 서북구,2,NaN,NaN,...,NaN,NaN,Chungcheongnam-do,"Seobuk-gu, Cheonan-si",NaN,121,184,20090825,NaN,441333253099


In [ ]:
# 도로명은 같지만 도로명번호, 도로명코드가 다른 경우 + 시군구코드가 다른 경우
df_case2_1 = df.loc[df.duplicated(subset='도로명', keep=False)].groupby(['도로명', '시군구코드', '도로명번호']).count().sort_values(by='도로명코드', ascending=False)
df_case2_1

읍면동일련번호  영문도로명  시도명  시군구명  읍면동구분  읍면동코드  읍면동명  사용여부  \
도로명      시군구코드 도로명번호                                                          
1100로    50110 3020003        1      1    1     1      1      0     0     1   
와룡로5길    46870 4694288        1      1    1     1      1      0     0     1   
와룡로174번길 44800 4586333        1      1    1     1      1      0     0     1   
와룡로2길    27290 4241422        1      1    1     1      1      0     0     1   
         46870 4694285        1      1    1     1      1      0     0     1   
...                         ...    ...  ...   ...    ...    ...   ...   ...   
백동길      46780 4670155        1      1    1     1      1      0     0     1   
         46790 4673168        1      1    1     1      1      0     0     1   
         46820 4682198        1      1    1     1      1      0     0     1   
         46840 4688159        1      1    1     1      1      0     0     1   
희망로3길    51720 4475668        1      1    1     1      1      0     0     1   

                        부여사유  변경이력사유  변경이력정보  영문시도명  영문시군구명  영문읍면동명  \
도로명      시군구코드 도로명번호                                                  
1100로    50110 3020003     1       0       0      1       1       0   
와룡로5길    46870 4694288     1       0       0      1       1       0   
와룡로174번길 44800 4586333     1       0       0      1       1       0   
와룡로2길    27290 4241422     1       0       0      1       1       0   
         46870 4694285     1       0       0      1       1       0   
...                      ...     ...     ...    ...     ...     ...   
백동길      46780 4670155     1       0       0      1       1       0   
         46790 4673168     1       0       0      1       1       0   
         46820 4682198     1       0       0      1       1       0   
         46840 4688159     1       0       0      1       1       0   
희망로3길    51720 4475668     1       0       0      1       1       0   

                        도로구간의시작지점기초번호  도로구간끝지점기초번호  효력발생일  말소일자  도로명코드  
도로명      시군구코드 도로명번호                                                    
1100로    50110 3020003              1            1      1     0      1  
와룡로5길    46870 4694288              1            1      1     0      1  
와룡로174번길 44800 4586333              1            1      1     0      1  
와룡로2길    27290 4241422              1            1      1     0      1  
         46870 4694285              1            1      1     0      1  
...                               ...          ...    ...   ...    ...  
백동길      46780 4670155              1            1      1     0      1  
         46790 4673168              1            1      1     0      1  
         46820 4682198              1            1      1     0      1  
         46840 4688159              1            1      1     0      1  
희망로3길    51720 4475668              1            1      1     0      1  

[43356 rows x 19 columns]

In [ ]:
df.loc[df['도로명']=='희망로3길'].sort_values(by='시군구코드')

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
27608,27200,4232378,00,희망로3길,Huimang-ro 3-gil,대구광역시,남구,2,NaN,NaN,...,NaN,NaN,Daegu,Nam-gu,NaN,1,62,20100420,NaN,272004232378
154509,51720,4475668,00,희망로3길,Huimang-ro 3-gil,강원특별자치도,홍천군,2,NaN,NaN,...,NaN,NaN,Gangwon-do,Hongcheon-gun,NaN,1,20,20091201,NaN,517204475668


In [ ]:
# 도로명은 같지만 도로명번호, 도로명코드가 다른 경우 + 시군구코드가 같은 경우
df_case2_2 = df.loc[df.duplicated(subset='도로명', keep=False)].groupby(['시군구코드', '도로명']).count().sort_values(by='도로명번호', ascending=False).head(20)
df_case2_2.loc[df_case2_2['도로명번호']>1]

,,도로명번호,읍면동일련번호,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,사용여부,부여사유,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
시군구코드,도로명,,,,,,,,,,,,,,,,,,,,
11230,한천로58길,2,2,2,2,2,2,0,0,2,1,0,0,2,2,0,1,1,1,0,2
51780,평화백마길,2,2,2,2,2,2,0,0,2,1,0,0,2,2,0,1,1,1,0,2
36110,소롱골길,2,2,2,2,0,2,0,0,2,2,0,0,2,0,0,2,2,2,0,2
43114,중앙로,2,2,2,2,2,2,0,0,2,1,0,0,2,2,0,2,2,1,0,2
36110,태산길,2,2,2,2,0,2,0,0,2,2,0,0,2,0,0,2,2,2,0,2
28110,미단희망로28번길,2,2,2,2,2,2,0,0,2,1,0,0,2,2,0,1,1,1,0,2
41800,북삼로80번가길,2,2,2,2,2,2,0,0,2,1,0,0,2,2,0,1,1,1,0,2
44131,휴대세교길,2,2,2,2,2,2,0,0,2,1,0,0,2,2,0,1,1,1,0,2
31110,풍암길,2,2,2,2,2,2,0,0,2,1,0,0,2,2,0,1,1,1,0,2


In [ ]:
df.loc[df['도로명']=='한천로58길']

,시군구코드,도로명번호,읍면동일련번호,도로명,영문도로명,시도명,시군구명,읍면동구분,읍면동코드,읍면동명,...,변경이력사유,변경이력정보,영문시도명,영문시군구명,영문읍면동명,도로구간의시작지점기초번호,도로구간끝지점기초번호,효력발생일,말소일자,도로명코드
14057,11290,4121702,00,한천로58길,Hancheon-ro 58-gil,서울특별시,성북구,2,NaN,NaN,...,NaN,NaN,Seoul,Seongbuk-gu,NaN,237,474,20100610,NaN,112904121702
98358,11230,4115640,00,한천로58길,Hancheon-ro 58-gil,서울특별시,동대문구,2,NaN,NaN,...,NaN,NaN,Seoul,Dongdaemun-gu,NaN,1,238,20100617,NaN,112304115640
145975,11230,4121702,00,한천로58길,Hancheon-ro 58-gil,서울특별시,동대문구,2,NaN,NaN,...,NaN,NaN,Seoul,Dongdaemun-gu,NaN,NaN,NaN,NaN,NaN,112304121702
